# Scraping: récupération des archives du journal Le Soir

## Imports

In [ ]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup

## Créer un répertoire pour stocker tous les articles

In [ ]:
txt_path = '../data/archives_lesoir'
# Créer le dossier s'il n'existe pas
if not os.path.exists(txt_path):
    os.mkdir(txt_path)

## Récupérer les URLs de 100 articles contenant le mot "linguistique"

Le journal [Le Soir](https://www.lesoir.be) met à disposition un moteur de recherche pour accéder à ses archives depuis 1989 (2 millions d'articles).
Il est possible de faire des recherches ciblées et de récupérer les articles répondant à cette recherche.

Essayons une recherche sur le thème "linguistique". Le journal renvoie 10 articles par page, mais nous nous focaliserons uniquement sur les articles payants. On va récupérer les URLs des articles sur les 100 premières pages, jusqu'à ce qu'on ait récupéré 100 urls.

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles_urls = list()
query = "linguistique"
for i in range(0, 100):
    print(f"Parsing page {i}")
    url = f"https://www.lesoir.be/archives/recherche?word={query}&sort=date%20desc&datefilter=anytime&start={i*10}"
    html = requests.get(url, headers=headers).text
    # Pattern pour extraire les urls des articles (/art/d-20220624-GVXEL9?)
    pattern = r"/art/[^\?]*"
    for art_url in re.findall(pattern, html):
        art_url = "https://www.lesoir.be" + art_url
        # Vérification que l'url n'est pas déjà dans la liste
        if art_url not in articles_urls:
            articles_urls.append(art_url)
    i += 10
    if len(articles_urls) >= 100:
        break
    print(f"{len(articles_urls)} articles trouvés")


In [ ]:
# Impression du nombre d'urls récupérées
print(len(articles_urls))
# Impression des 10 premières URLs
print(articles_urls[:10])


## Télécharger tous les articles et imprimer le titre, le chapô et la partie de l'article visible sans abonnement dans un fichier texte

In [ ]:
for url in articles_urls:
    # Récupération de l'id de l'article sur la base de son URL
    article_id = url.split("/")[-1]
    # Création d'un nom de fichier et vérification qu'il n'existe pas déjà
    filename = os.path.join(txt_path, article_id + ".txt")
    if not os.path.exists(filename):
        # Téléchargement de l'article et parsing du HTML
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, 'html.parser')
        # Récupération du titre, du chapô et du contenu visible
        title = soup.find("h1").get_text(separator=' ')
        chapo = soup.find("r-article--chapo").get_text(separator=' ')
        content = soup.find("r-article--section").get_text(separator=' ')
        # Ecriture du fichier
        with open(filename, 'w') as f:
            f.write(f"{title}\n\n{chapo}\n\n{content}")
    else:
        print(f"{article_id}: déjà téléchargé")
    

## Vérifier que tous les articles ont été téléchargés


Si ce n'est pas le cas, vous pouvez relancer l'étape de téléchargement (elle ignorera les documents déjà téléchargés)

In [ ]:
ok_count = 0
for url in articles_urls:
    filename = url.split("/")[-1]
    downloads = os.listdir(txt_path)
    if f"{filename}.txt" not in downloads:
        print(f"{filename} is missing!")
    else:
        ok_count += 1
print(f"{ok_count} files found out of {len(articles_urls)}!")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp